Todo: sloučit povídky povídkových filmů do 1 řádku?

In [1]:
import re
import os
import json
import pandas as pd

In [2]:
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())

In [3]:
with open(os.path.join("data_fixes", "seznamy.json"), encoding="utf-8") as seznamy:
    seznamy = json.loads(seznamy.read())
profese = seznamy["profese"]

In [4]:
df = pd.read_json(os.path.join("data", "filmy_raw.json"))

## Co do seznamu

In [5]:
seznamove_sloupce = [
    "Animace",
    "Animační technika",
    "Architekt",
    "Asistent kamery",
    "Asistent režie",
    "Asistent střihu",
    "Asistent vedoucího produkce",
    "Asistent vedoucího výroby",
    "Asistent zvuku",
    "Assistent architekta",
    "Ateliéry",
    "Autor komentáře",
    "Autor projektu",
    "Barva",
    "České podtitulky",
    "Dialogy české verze",
    "Distribuční nosič",
    "Dramaturg",
    "Druhá kamera",
    "Hudba",
    "Choreografie",
    "Kamera",
    "Kaskadéři",
    "Koordinátor kaskadérů",
    "Koproducent",
    "Kostýmy",
    "Masky",
    "Mezititulky",
    "Mluveno",
    "Natáčecí formát",
    "Návrhy kostýmů",
    "Návrhy titulků",
    "Odborný poradce",
    "Pedagogické vedení",
    "Podtitulky",
    "Pomocná režie",
    "Postprodukce",
    "Producent",
    "Produkce",
    "Předloha",
    "Původní filmový námět",
    "Režie",
    "Režie české verze",
    "Scénář",
    "Skript",
    "Spolupráce",
    "Střih",
    "Technický scénář",
    "Typologie",
    "Úvodní/závěrečné titulky",
    "Vedoucí produkce",
    "Vedoucí výroby",
    "Verze",
    "Výprava",
    "Výkonná produkce",
    "Výtvarná spolupráce",
    "Výtvarník",
    "Zástupce vedoucího produkce",
    "Zástupce vedoucího výroby",
    "Země původu",
    "Zpívá",
    "Zvuk",
    "Zvukový systém/formát",
    "Žánr"]

In [6]:
seznamove_sloupce_firmy = [
    "Distribuce",
    "Nositelé copyrightu",
    "Partner",
    "S podporou",
    "Triky",
    "Vizuální efekty",
    "Výrobce",
    "Zvláštní efekty"
]

In [7]:
jiz_seznam = [
    "Dirigent",
    "Hudba k písni",
    "Nahrál",
    "Písně",
    "Použitá hudba",
    "Tančí",
    "Text písně",
    "Hudební dramaturgie",
    "Hudební režie",
    "Výběr hudby",
    "Zpěv role"
]

## Funkce pro čištění

In [8]:
def nahrad(bunka, original, nahrada):
    
    if isinstance(bunka, list):
        opravena_bunka = []
        try:
            for b in bunka:
                if b == original:
                    opravena_bunka.append(nahrada)
                elif original in b:
                    opravena_bunka.append(b.replace(original,nahrada))
                else:
                    opravena_bunka.append(b)
        except:
            opravena_bunka = bunka
            
    elif isinstance(bunka, str):
        opravena_bunka = ""
        if bunka == original:
            opravena_bunka = nahrada
        elif original in bunka:
            opravena_bunka = bunka.replace(original, nahrada)
        else:
            opravena_bunka = bunka
    
    else:
        opravena_bunka = bunka
    
    return opravena_bunka

In [9]:
def citlivy_orez(x):
    if isinstance(x, str):
        x = x.strip()
    if isinstance(x, list):
        try:
            x = [y.strip() for y in x]
        except:
            pass
    return(x)

In [10]:
def do_seznamu(polozka):
    try:
        if "," in polozka:
            polozka = str(polozka).split(",")
            polozka = [p.strip() for p in polozka]
    except:
        pass
    return(polozka)

In [11]:
def firmy_do_seznamu(polozka):
    ## a tituly
    if polozka:
        polozka = re.sub("\s"," ",polozka)
        polozka = re.sub("\n"," ",polozka)
        polozka = polozka.replace(" spol. s "," s. ").replace(" spol.s "," s. ").replace("s.r.o.","s. r. o.").replace("s. r.o.","s. r. o.").replace("a.s.","a. s.").replace(", s. r. o."," s. r. o.").replace(",s. r. o."," s. r. o.").replace(", a. s."," a. s.").replace(",a. s."," a. s.").replace(", v. o. s."," v. o. s.").replace(", CSc.","").replace(", DrSc.","")
        polozka = do_seznamu(polozka)
    else:
        polozka = polozka
    return(polozka)

## Čištěníčko

In [12]:
df["Režie"] = df["Režie"].apply(lambda x: nahrad(x,"české verze", ""))

### Pryč s duplikáty

In [13]:
df = df.drop_duplicates(subset="Film")

### Ořez

In [14]:
df = df.apply(lambda x: citlivy_orez(x))

### Vytvoření seznamů

In [15]:
for s in seznamove_sloupce:
    df[s] = df[s].apply(lambda x: do_seznamu(x))
for s in seznamove_sloupce_firmy:
    df[s] = df[s].apply(lambda x: firmy_do_seznamu(x))

### Lokace, poměr stran, metráž atd.

In [16]:
def seznam_lokaci(lokace_hrube):
    lokace_ciste = []
    if lokace_hrube:
        lokace_hrube = lokace_hrube.split("),")
        for x in lokace_hrube:
            if "(" in x:
                x = x + ")"
            x = x.strip()
            x = x.replace("))",")")
            if x[0:2] == ", ":
                x = x[2:]
            if x != ")":
                lokace_ciste.append(x)
    return(lokace_ciste)

In [17]:
df["Lokace"] = df["Lokace"].apply(lambda x: seznam_lokaci(x))

In [18]:
df["Minutáž"] = df["Minutáž"].apply(lambda x: str(x).replace("min","").strip())
df["Minutáž"] = pd.to_numeric(df["Minutáž"], errors="coerce")

In [19]:
def ratio(x):
    if x:
        x = x.replace("1:","")
        x = x.split(", ")
        x = [y.strip() for y in x]
        x = [y.replace(",",".") for y in x]
        x = [pd.to_numeric(y) for y in x]
        if len(x) == 1:
            x = x[0]
    return(x)

In [20]:
df["Poměr stran"] = df["Poměr stran"].apply(lambda x: ratio(x))

In [21]:
def metraz(x):
    if x:
        x = int(x.replace("metrů","").replace(" ",""))
    else:
        x = x
    return x

In [22]:
df["Původní metráž"] = df["Původní metráž"].apply(lambda x: metraz(x))

In [23]:
def kill_whitespace(x):
    def seriously(y):
        y = re.sub("\s+", " ", y)
        return(y)
    if isinstance(x, list):
        x = [seriously(y) for y in x]
    if isinstance(x, str):
        x = seriously(x)
    return(x)

In [24]:
df["Ocenění"] = df["Ocenění"].apply(lambda x: kill_whitespace(x))

In [25]:
def roky_vyroby(x):
    if x:
        zacatek = int(x[:4])
        konec = int(x[-4:])
    else:
        zacatek = None
        konec = None
    return [zacatek, konec]

In [26]:
df["Začátek výroby"] = df["Rok výroby"].apply(lambda x: roky_vyroby(x)[0])
df["Konec výroby"] = df["Rok výroby"].apply(lambda x: roky_vyroby(x)[1])

C:\Users\micha\AppData\Local\Temp\ipykernel_25556\3186634046.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Konec výroby"] = df["Rok výroby"].apply(lambda x: roky_vyroby(x)[1])


### Seřadit podle copyrightu

In [27]:
df = df.sort_values(by = "Copyright")

### Oprava minutáže

In [28]:
df[df["Minutáž"] > 130].groupby("Film")["Minutáž"].max().nlargest(60).index.to_list()

['Místo v houfu (396595)',
 'Hrdina divokého západu (402185)',
 'Zapomenuté transporty (79193)',
 'Komunismus a síť, aneb Konec zastupitelské demokracie (402026)',
 'Bohemia docta aneb Labyrint světa a lusthauz srdce /Božská komedie/ (17002)',
 'Kdo bude hlídat hlídače? Dalibor aneb Klíč k Chaloupce strýčka Toma (20716)',
 'Dny zrady (396979)',
 'Venoušek a Stázička (395308)',
 'Co dělat? Cesta z Prahy do Českého Krumlova aneb Jak jsem sestavoval novou vládu (9101)',
 'Nový Hyperion aneb Volnost, rovnost, bratrství (7145)',
 'Hořící keř (398412)',
 '25 ze šedesátých aneb Československá nová vlna (158554)',
 'Tmář a jeho rod aneb Slzavé údolí pyramid (80194)',
 'Lazebník sibiřský (24194)',
 'Vojta Lavička: Nahoru a dolů (103360)',
 'Nabarvené ptáče (401962)',
 'Babička I., II. (396899)',
 'Marketa Lazarová (396753)',
 'Zviditelnění (397888)',
 'Ragtime (401385)',
 'Amadeus (401253)',
 'Nejasná zpráva o konci světa (9463)',
 'Baader Meinhof Komplex (99894)',
 'Želary (13452)',
 'Putování

- Místo v houfu (396595) – Zjevná chyba, nahradíme údajem z ČSDF.
- Venoušek a Stázička, Vojta Lavička: Nahoru a dolů (103360) – dtto
- Hrdina divokého západu (402185) – Složitější: "Humorné příhody Eddieho Pola a Marie Walcampové v „15 epizodách a 142 dílech“. Nejméně špatné ze špatných řešení: nahradíme mediánem za rok 1920, takže to nerozdrbe statistiky.
- Zapomenuté transporty (79193) – vyhazujeme, nechávám jednotlivé díly.

In [29]:
df.loc[df["Film"] == "Místo v houfu (396595)","Minutáž"]=115
df.loc[df["Film"] == "Venoušek a Stázička (395308)","Minutáž"]=68
df.loc[df["Film"] == "Vojta Lavička: Nahoru a dolů (103360)","Minutáž"]=89
df.loc[df["Film"] == "Hrdina divokého západu (402185)","Minutáž"]=df[df["Copyright"] == 1920]["Minutáž"].median().astype(int)
df = df[~(df["Film"] == "Zapomenuté transporty (79193)")]

In [30]:
df = df.drop(labels="cyklus", axis=1)

### Odstranění zahraničních produkcí, exilových filmů M. Formana a bondovky

In [31]:
zahranicni = ["Humorista (401985)","Útěk (401824)","Amundsen (401963)","Oliver Twist (37449)","Nexus (7784)"]

In [32]:
df = df[~df.Film.isin(opravy["exil"])]
df = df[~df.Film.isin(zahranicni)]
df = df[df["Země původu"].str.contains("Česk|Rakousko-Uhersko",na=True)]

## Odstranění druhých závorek

In [33]:
df["Vedoucí produkce"].drop_duplicates().to_list()

[None,
 ['3534', ''],
 ['4473', ''],
 ['5420', ''],
 ['27426', ''],
 ['4450', ''],
 ['74907', '3334', ''],
 ['34813', '1143', 'Zdenka Deitchová (kreslená část)'],
 ['34813', ''],
 ['85990', 'Šárka Cimbalová (vedoucí natáčení)'],
 ['', 'Jana Novosadová (vedoucí natáčení)'],
 ['', 'Jaroslav Kučera (vedoucí natáčení)'],
 ['9671', ''],
 ['9232', 'Luboš Tůma (vedoucí atelieru)'],
 ['3246', ''],
 ['12599', 'Antonín Suchánek (vedoucí natáčení)'],
 ['7470', 'Jindřich Bareš (ČT)'],
 ['2388',
  'Markéta Hájková (vedoucí natáčení)',
  'Andrea Ashton /ž/ (vedoucí natáčení)'],
 ['147', ''],
 ['1349', ''],
 ['644', '34483', 'Ľubomír Matušek'],
 ['2808', ''],
 ['7093', ''],
 ['6996', '2106', ''],
 ['7697', ''],
 ['1768', '1769', ''],
 ['6711', ''],
 ['2507', ''],
 ['', 'Radmila Haertová'],
 ['4013', ''],
 ['658', ''],
 ['34483', 'Ivan Nagy (vedoucí natáčení)'],
 ['2808', '1338', 'Martin Kopřiva (ČT)'],
 ['644', ''],
 ['3007', '658', 'Alexej Guha (ČT)'],
 ['4813', '1245', 'Darina Levová (ČT)'],
 ['682

In [34]:
def odzavorkuj(bunka):
    
    def odzav(x):
        return re.sub("\(.*[^\d]\)","", x)
    
    if isinstance(bunka, list):
        cista_bunka = [odzav(x).strip() for x in bunka if x]
            
    elif isinstance(bunka, str):
        cista_bunka = odzav(bunka).strip()
        
    else:
        cista_bunka = bunka
        
    return cista_bunka

In [35]:
for p in profese:
    df[p] = df[p].apply(odzavorkuj)

In [36]:
df["Vedoucí produkce"].drop_duplicates().to_list()

[None,
 ['3534'],
 ['4473'],
 ['5420'],
 ['27426'],
 ['4450'],
 ['74907', '3334'],
 ['34813', '1143', 'Zdenka Deitchová'],
 ['34813'],
 ['85990', 'Šárka Cimbalová'],
 ['Jana Novosadová'],
 ['Jaroslav Kučera'],
 ['9671'],
 ['9232', 'Luboš Tůma'],
 ['3246'],
 ['12599', 'Antonín Suchánek'],
 ['7470', 'Jindřich Bareš'],
 ['2388', 'Markéta Hájková', 'Andrea Ashton /ž/'],
 ['147'],
 ['1349'],
 ['644', '34483', 'Ľubomír Matušek'],
 ['2808'],
 ['7093'],
 ['6996', '2106'],
 ['7697'],
 ['1768', '1769'],
 ['6711'],
 ['2507'],
 ['Radmila Haertová'],
 ['4013'],
 ['658'],
 ['34483', 'Ivan Nagy'],
 ['2808', '1338', 'Martin Kopřiva'],
 ['644'],
 ['3007', '658', 'Alexej Guha'],
 ['4813', '1245', 'Darina Levová'],
 ['6827'],
 ['1768', '5973', 'Jiří Koštýř'],
 ['1799'],
 ['5284'],
 ['7300'],
 ['12111'],
 ['8666', '9228', '11155', '9227', 'Otakar Svoboda'],
 ['16517', '644', 'Miroslav Fedor', 'Aleš Novák'],
 ['21405'],
 ['6352', '13649'],
 ['2046'],
 ['89423'],
 ['1691', '5973', 'Jiří Koštýř'],
 ['7406', 

## Opravy jmen

In [37]:
def replace_names(x):
    if isinstance(x, str):
        if x.strip() in ids_jmena:
            return ids_jmena[str(x)]
        else:
            return x
    elif isinstance(x, list):
        return [str(replace_names(xi)) for xi in x]
    else:
        return x

### Dočištění duplicit

In [38]:
ids_jmena = opravy["jmena"]
ids_jmena

{'Otakar Vávra': '9507',
 'J. A. Holman': '37290',
 'Hrušínský Rudolf ml.': '2988',
 'Rudolf Hrušínský': '2988',
 'František Michálek': '34045',
 'Jan Kališ': '2009',
 'Alois Fišárek': '570',
 'Bohumil Šmída': '31114',
 'Ivo Marák': '22203',
 'Alena Müllerová': '11622',
 'Antonín Frič': '127707',
 'Willy Ströminger': '55487',
 'Karel Ješátko': '45886',
 'Jiří Koštýř': '5973',
 'Zdeněk Hala': '29088',
 'Tomáš Šimon': '45286',
 'Jaroslav Kučera': '1647',
 'Jiří Vlach': '9748',
 'Ivana Kurincová': '84166',
 'Aleš Najbrt': '5647',
 'Jan Poukar': '117876',
 'Zuzana Vrbová': '7187',
 'Lucie Zvěřinová': '24395',
 'Bohumír Brunclík': '126058',
 'Anna Kopecká': '76947',
 'Artemio Benki': '1906',
 'Wally V. Kolman': '46614',
 'Waly V. Kolman': '46614',
 'Antonie Dědečková': '123312',
 'Fero Fenič': '608',
 'Miloslav Mirvald': '21426',
 'Kateřina Kovářová': '88592',
 'Jiří Stach': '34555'}

In [39]:
for p in profese:
    df[p] = df[p].apply(replace_names)

### Nahrazení ids jmény

In [40]:
persony = pd.read_json(os.path.join("data", "persony.json"))

In [41]:
persony = persony[["Id","Jméno"]]

In [42]:
persony.sample(3)

,Id,Jméno
29454,80609,Tomáš Tožička (80609)
22094,5213,Libo Suembun (5213)
15017,2656,Arvo Kukumägi (2656)


In [43]:
persony[persony["Id"] == "person"]

,Id,Jméno
35812,person,Čechomor (person)
35813,person,Kevin Mc Kidd (person)
35814,person,Rostislav Novák st. (person)


Tohle nás asi nemusí trápit.

In [44]:
ids_jmena = {}
for index, row in persony.iterrows():
    ids_jmena[str(row["Id"])] = str(row["Jméno"])
ids_jmena

{'1000': 'Ondřej Mašek (1000)',
 '100019': 'kino Světozor (100019)',
 '100038': 'AD WOOD, s.r.o. (100038)',
 '100039': 'Hypermarket Film s.r.o. (100039)',
 '100076': 'Punk Film, s.r.o. (100076)',
 '10009': 'Jan Kašpar (10009)',
 '1001': 'Renata Pavlíčková (1001)',
 '10010': 'Aleš Zábojník (10010)',
 '100116': 'Tomáš Dalecký (100116)',
 '100137': 'FILM FACTORY, s.r.o. (100137)',
 '100138': 'FILM FACTORY CZ, s.r.o. (100138)',
 '100139': 'Vojtěch Matyáš Jírovec (100139)',
 '100156': 'Josef Gabriel (100156)',
 '100196': 'Pegasos Film (100196)',
 '100197': 'SisaArt (100197)',
 '1002': 'Jakub Zich (1002)',
 '100216': 'Asociácia slovenských filmových klubov – ASFK (100216)',
 '100219': 'Starorežná Prostějov, k.s. (100219)',
 '100221': 'Agentura Tomáš Zůbek (100221)',
 '100236': 'Radka Production s.r.o. (100236)',
 '100237': 'MAUR film s.r.o. (100237)',
 '10026': 'Miroslav Kochánek (10026)',
 '100262': 'Here and Now Production (100262)',
 '100263': 'Open City Films (100263)',
 '10027': 'Michal

In [45]:
for p in profese:
    df[p] = df[p].apply(replace_names)

### Mazání titulů

In [46]:
tituly = [", CSc.",", DrSc."]

In [47]:
for p in profese:
    for t in tituly:
        df[p] = df[p].apply(lambda x: nahrad(x, t, ""))

### Mazání "/ž/"

In [48]:
def odzen(bunka):
    
    def odz(x):
        return x.replace(" /ž/","").strip()
    
    if isinstance(bunka, list):
        try:
            cista_bunka = [odz(x) for x in bunka]
        except:
            cista_bunka = bunka
            
    if isinstance(bunka, str):
        cista_bunka = odz(bunka)
        
    else:
        cista_bunka = bunka
        
    return cista_bunka

In [49]:
for p in profese:
    df[p] = df[p].apply(lambda x: nahrad(x, " /ž/", ""))

## Kontrola, export

In [50]:
df.sample(5)

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,Začátek výroby,Konec výroby
2609,Upír z Feratu (397313),Československo,1981,1981,premiéra 28. 5. 1982 /nepřístupný mládeži/,91.0,[Juraj Herz (7875)],film,"[horor, sci-fi]","[hraný, distribuční]",...,"Dramaturgická skupina Drahoslava Makovičky, Dr...",None,None,None,None,None,None,None,1981,1981
1233,Turbina (395935),Československo,1941,1941,premiéra 17. 10. 1941 /nepřístupný mládeži/ (k...,98.0,[Otakar Vávra (9507)],film,drama,"[hraný, distribuční]",...,None,None,None,None,None,None,None,None,1941,1941
1912,Láhve (396616),Československo,1964,1964,premiéra 6. 11. 1964 /přístupný mládeži/ (celo...,19.0,[Bohumil Svoboda (12732)],film,"[dětský, moralita]","[hraný, distribuční]",...,None,None,None,None,None,None,None,None,1964,1964
2060,Guide (396764),Československo,1968,1968,None,26.0,[Iskra Dimčevska-Ouhilal (128956)],film,příběh,"[hraný, nedistribuční, studentský]",...,None,None,None,None,None,None,None,None,1968,1968
3586,Proroci a básníci. Kapitoly z kalendáře (400741),Česká republika,2000,2000,"premiéra 9. 11. 2000 /přístupný/ (kino Aero, P...",88.0,[Ivan Vojnár (4787)],film,dokument,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,2000,2000


In [51]:
df.shape

(4506, 133)

In [52]:
df.to_json(os.path.join("data", "filmy.json"), orient="records")

In [53]:
import zipfile
try:
    import zlib
    mode= zipfile.ZIP_DEFLATED
except:
    mode= zipfile.ZIP_STORED